In [1]:
from utils import read_json, query_route, add_plate_numbers_to_df
import googlemaps
import pandas as pd
from datetime import datetime

# GOOGLE_API_KEY = open('google_maps_key', 'r').readlines()[0]
# gmaps = googlemaps.Client(key=GOOGLE_API_KEY)

BQ_API_KEY = open('google_bq_key', 'r').readlines()[0]
MAPBOX_API_KEY = open('mapbox_key', 'r').readlines()[0]



# Idea Backlog
* If we query mapbox for all routes at the start of the hour, we'll have really bad performance at the start then good performance later on; what if we iteratively query throughout the hour, while using historical data for the rest
* Moreover, we shouldn't query ALL jeeps to ALL stops in the route, cuz that will cost lots of money
    * What we should do instead is figure out which stop it's closest to, and query that

Hint: The database table needs to look like this

|StopID|List of Jeeps|List of Arrival Times|List of Jeep Routes|
|-----|-----|-----|-----|
|AL1|ID1, ID2|5.5, 10|Aurora Loop, Aurora Loop|
|AL2|ID3, ID1|6, 20|Aurora Loop, Aurora Loop|
|QA1|ID4, ID5|10, 20|Quezon Ave Loop, Quezon Ave Loop|
|QA2|ID6, ID4|10, 20|Quezon Ave Loop, Quezon Ave Loop|

In [2]:
stop_coords_mapping_dict = read_json("data/stop_coords_mapping_dict.json")
jeep_route_mapping_dict = read_json("data/jeep_route_mapping_dict.json")
jeep_information_dict = read_json("data/jeep_information_dict.json")
route_stops_mapping_dict = read_json("data/route_stops_mapping_dict.json")


In [3]:
historical_geocoding_table = pd.DataFrame({
    "names": ["A", "B", "C"],
    "lng": [121.5, 121.25, 121.0],
    "lat": [14.5, 14.25, 14.0]
})

historical_eta_table = pd.DataFrame({
    "stop_id": ["AL1", "AL1", "AL1", "AL2", "AL2"], 
    "lng": [121.98, 121.25, 121.66, 121.23, 121.95], 
    "lat": [14.23, 14.74, 14.63, 14.33, 14.04],
    "time": [datetime.now()]*5, 
    "eta": [5.52, 7.84, 2.25, 11.54, 4.25]
})


In [4]:
temp = query_route(
    route_name = "aurora_loop",
    jeep_route_mapping_dict = jeep_route_mapping_dict, 
    route_stops_mapping_dict = route_stops_mapping_dict, 
    stop_coords_mapping_dict = stop_coords_mapping_dict, 
    historical_geocoding_table = historical_geocoding_table,
    historical_eta_table = historical_eta_table,
    use_mapbox = False,
    MAPBOX_API_KEY = MAPBOX_API_KEY
    )
historical_geocoding_table = temp.pop("historical_geocoding_table")
historical_eta_table = temp.pop("historical_eta_table")

[[121.02612759  14.66907581]
 [121.04331558  14.6681457 ]
 [121.03992056  14.6963531 ]]
[[121.1234  14.5678]
 [121.2341  14.6785]
 [121.3412  14.7856]
 [121.4123  14.8567]
 [121.4321  14.8765]]


In [5]:
df_temp = pd.DataFrame(temp)

In [6]:
df_temp["jeep_plate_num_list"] = add_plate_numbers_to_df(df_temp["jeep_ids_list"], jeep_information_dict)

In [7]:
df_temp

,stop_id_list,jeep_ids_list,jeep_locations_list,jeep_arrival_times_list,jeep_plate_num_list
0,AL1,"[ID2, ID1, ID3]","[B, B, B]","[5.509503660332623, 5.5255688341529, 5.5195940...","[DEF67890, ABC12345, GHI12345]"
1,AL2,"[ID2, ID3, ID1]","[B, B, B]","[9.67457419360832, 9.703916483216936, 9.709461...","[DEF67890, GHI12345, ABC12345]"
2,AL3,"[ID3, ID2, ID1]","[B, B, B]","[-1.0, -1.0, -1.0]","[GHI12345, DEF67890, ABC12345]"
3,AL4,"[ID3, ID2, ID1]","[B, B, B]","[-1.0, -1.0, -1.0]","[GHI12345, DEF67890, ABC12345]"
4,AL5,"[ID3, ID2, ID1]","[B, B, B]","[-1.0, -1.0, -1.0]","[GHI12345, DEF67890, ABC12345]"
